# Introduction to Machine Learning - The Final Model
#### By Jeroen Smienk & Marnick van der Arend

In [1]:
# IMPORTS AND NOTEBOOK SETUP
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
# Importing the dataset
data = pd.read_csv('../dataset-numpy/dataset-v6.csv')
data.columns

Index([u'area', u'contours', u'radius', u'hull_radius', u'centroid_x',
       u'centroid_y', u'angle', u'weight_0_2', u'weight_0_3', u'weight_0_4',
       u'weight_0_5', u'weight_0_6', u'weight_1_1', u'weight_1_2',
       u'weight_1_3', u'weight_1_4', u'weight_1_5', u'weight_1_6',
       u'weight_2_1', u'weight_2_2', u'weight_2_3', u'weight_2_4',
       u'weight_2_5', u'weight_2_6', u'weight_3_1', u'weight_3_2',
       u'weight_3_3', u'weight_3_4', u'weight_3_5', u'weight_3_6',
       u'weight_4_1', u'weight_4_2', u'weight_4_3', u'weight_4_4',
       u'weight_4_5', u'weight_4_6', u'weight_5_1', u'weight_5_2',
       u'weight_5_3', u'weight_5_4', u'weight_5_5', u'weight_5_6',
       u'weight_6_1', u'weight_6_2', u'weight_6_3', u'weight_6_4',
       u'weight_6_5', u'weight_6_6', u'weight_7_2', u'weight_7_3',
       u'weight_7_4', u'weight_7_5', u'weight_7_6', u'weight_7_7',
       u'num_holes', u'label'],
      dtype='object')

## Normalization

In [3]:
# Normalization
columns = data.columns.values

columns_to_not_normalize = ['num_holes', 'label']
columns_to_normalize = [c for c in columns if not c in columns_to_not_normalize]

# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[columns_to_normalize] = pd.DataFrame(scaler.fit_transform(data[columns_to_normalize]), columns=columns_to_normalize)

## Training the classifier


In [5]:
# SVM
from sklearn import svm

# Splitting our data
X_train, Y_train = data.iloc[:,:-1], data.iloc[:,-1]

classifier = svm.SVC(kernel='rbf', C=3.8, gamma=0.1)
classifier.fit(X_train, Y_train)
# print("Score: {}".format(classifier.score(X_test, Y_test)))

SVC(C=3.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Exporting


In [6]:
# Exporting the scaler and classifier
from sklearn.externals import joblib

joblib.dump(scaler, 'scaler_svm_v6.joblib') 
joblib.dump(classifier, 'classifier_svm_v6.joblib') 

['classifier_svm_v6.joblib']